In [89]:
import requests
import time
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool
import threading
from functools import wraps
import concurrent.futures
requests.packages.urllib3.disable_warnings()

In [90]:
urls = ['https://www.northwestknowledge.net/metdata/data/pr_1979.nc',
'https://www.northwestknowledge.net/metdata/data/pr_1980.nc',
'https://www.northwestknowledge.net/metdata/data/pr_1981.nc',
'https://www.northwestknowledge.net/metdata/data/pr_1982.nc']

In [91]:
fns = [r'C:\Users\zhemati\Documents\git_zhf\downloads\pr_1979.nc',
r'C:\Users\zhemati\Documents\git_zhf\downloads\pr_1980.nc',
r'C:\Users\zhemati\Documents\git_zhf\downloads\pr_1981.nc',
r'C:\Users\zhemati\Documents\git_zhf\downloads\pr_1982.nc']

In [92]:
inputs = zip(urls, fns)

In [93]:
def download_url(args):
    t0 = time.time()
    url, fn = args[0], args[1]
    try:
        r = requests.get(url)
        with open(fn, 'wb') as f:
            f.write(r.content)
        return(url, time.time() - t0)
    except Exception as e:
        print('Exception in download_url():', e)

In [77]:
t0 = time.time()
for i in inputs:
    result = download_url(i)
    print('url:', result[0], 'time:', result[1])
print('Total time:', time.time() - t0)

url: https://www.northwestknowledge.net/metdata/data/pr_1979.nc time: 7.025955438613892
url: https://www.northwestknowledge.net/metdata/data/pr_1980.nc time: 8.76747441291809
url: https://www.northwestknowledge.net/metdata/data/pr_1981.nc time: 7.602313280105591
url: https://www.northwestknowledge.net/metdata/data/pr_1982.nc time: 7.201810836791992
Total time: 30.620519161224365


In [94]:
def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        threading.current_thread().thread_duration = end_time - start_time
        print(f'Function {func.__name__}{args} Took {total_time:.4f} seconds')
        return result
    return timeit_wrapper

In [95]:
if __name__ == "__main__":
    
    # Capture program start time
    start_time = time.perf_counter()
    start = time.strftime("%d.%m.%Y %H:%M:%S")
    
    thread_pool = list()  # container to hold threads
    duration_from_decorator = 0



    for i in inputs:
        wrapped_worker = timeit(download_url)
        t = threading.Thread(target=wrapped_worker, args=[i])
        thread_pool.append(t)

    for _thread in thread_pool:
        _thread.start()


    for _thread in thread_pool:
        _thread.join()
    
    # Capture program end time
    end_time = time.perf_counter()
    stop = time.strftime("%d.%m.%Y %H:%M:%S")
    execution_time = end_time - start_time

    print('start:', start, '', 'stop:', stop)
    print('Total execution time:', execution_time)
    print('Decorated Threads total duration:', duration_from_decorator)    

Function download_url(('https://www.northwestknowledge.net/metdata/data/pr_1981.nc', 'C:\\Users\\zhemati\\Documents\\git_zhf\\downloads\\pr_1981.nc'),) Took 12.0132 seconds
Function download_url(('https://www.northwestknowledge.net/metdata/data/pr_1980.nc', 'C:\\Users\\zhemati\\Documents\\git_zhf\\downloads\\pr_1980.nc'),) Took 16.2444 seconds
Function download_url(('https://www.northwestknowledge.net/metdata/data/pr_1979.nc', 'C:\\Users\\zhemati\\Documents\\git_zhf\\downloads\\pr_1979.nc'),) Took 19.3180 seconds
Function download_url(('https://www.northwestknowledge.net/metdata/data/pr_1982.nc', 'C:\\Users\\zhemati\\Documents\\git_zhf\\downloads\\pr_1982.nc'),) Took 20.1854 seconds
start: 23.11.2022 15:40:14  stop: 23.11.2022 15:40:34
Total execution time: 20.237335000012536
Decorated Threads total duration: 0
